In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
from codec import ImageCompressor

In [13]:
min_sparcity = 100
min_n = 2**5
max_n = 2**5
a_cols = 256
fif_version = 2
magic_number = b'FIF'  # Ensure this is a bytes object
header_format = '3sBiiBBBBB'
v_format_precision = "f" 
max_error = 0.1

newCodec = ImageCompressor(min_sparcity,
                           min_n,
                           max_n,
                           a_cols,
                           fif_version,
                           magic_number,
                           header_format,
                           v_format_precision,
                           max_error
                          )

In [ ]:
input_file = 'lena.png'
output_file = 'lena_compressed.fif'

import warnings

with warnings.catch_warnings():
    #Following RuntimeWarning ignored:
    #Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
    warnings.simplefilter("ignore", RuntimeWarning)
    
    #Encode image
    newCodec.code(input_file, output_file)

In [ ]:
input_file = 'lena_compressed.fif'
output_file = 'lena_reconstructed.png'
newCodec.decode(input_file, output_file)

In [ ]:
from skimage import io, img_as_float
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
path = ""
image1 = img_as_float(io.imread(path + 'lena.png', as_gray=True))
image2 = img_as_float(io.imread('lena_reconstructed.png', as_gray=True))

if image1.shape != image2.shape:
    raise ValueError("Input images must have the same dimensions.")

data_range = image1.max() - image1.min()

ssim_index, ssim_image = ssim(image1, image2, full=True, data_range=data_range)

print(f"SSIM Index: {ssim_index}")

plt.imshow(ssim_image, cmap='gray')
plt.title('SSIM Map')
plt.colorbar()
plt.show()